In [60]:
import os
import re
import json
import argparse
from pathlib import Path
from typing import List, Tuple, Dict, Optional
from openai import OpenAI

In [61]:
_HEADING_RE = re.compile(r'^(#{1,6})\s*(.+?)\s*#*\s*$')  # ATX 헤딩: # ... ######
_FENCE_RE = re.compile(r'^(```+|~~~+)')                  # 코드펜스 토글

def extract_all_sections_to_json(md_path: str) -> str:
    """
    마크다운 파일에서 모든 섹션(#~######)을 추출하여
    {"sections": [{"section_title": "<헤딩 원문>", "text": "<다음 헤딩 전까지 본문>"}]} 형태의
    JSON 문자열을 반환합니다.

    - 코드펜스 내부의 헤딩 표시는 무시합니다.
    - 헤딩 이전의 프롤로그 텍스트는 섹션으로 취급하지 않습니다.
    - section_title은 마크다운의 원문 헤딩 텍스트(샾 제외)를 그대로 보존합니다.
    """
    with open(md_path, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    sections = []
    current = None
    in_code = False

    for line in lines:
        # 코드펜스 토글
        if _FENCE_RE.match(line):
            in_code = not in_code
            # 본문으로는 그대로 저장 (현재 섹션이 열려 있으면)
            if current:
                current["text"].append(line)
            continue

        # 코드블록 밖에서만 헤딩 인식
        if not in_code:
            m = _HEADING_RE.match(line)
            if m:
                # 이전 섹션 마감
                if current:
                    current["text"] = "".join(current["text"]).rstrip()
                    sections.append(current)
                # 새 섹션 시작
                title = m.group(2).strip()
                current = {"section_title": title, "text": []}
                continue

        # 일반 본문 라인 기록
        if current:
            current["text"].append(line)

    # 마지막 섹션 마감
    if current:
        current["text"] = "".join(current["text"]).rstrip()
        sections.append(current)

    return json.dumps({"sections": sections}, ensure_ascii=False, indent=2)

In [62]:
json_str = extract_all_sections_to_json(md_path="out/korean_full.md")
print(json_str)

{
  "sections": [
    {
      "section_title": "Using Scenario-Writing for Identifying and Mitigating Impacts of Generative AI",
      "text": ""
    },
    {
      "section_title": "ABSTRACT",
      "text": "영향평가는 AI 배치의 부정적·긍정적 함의를 규명하고 그 사용의 부작용을 회피하기 위한 일반적인 수단으로 등장했다. 영향평가가 중요하다는 사실은 부인할 수 없으며, 특히 생성형 AI와 같이 급속히 확산되는 기술의 경우 더욱 그렇다. 그러나 현재의 영향평가 문헌과 관행을 비판적으로 검토하여 그 한계를 규명하고, 이러한 한계에 대응하는 새로운 접근법을 개발하는 것도 필수적이다. 이 글에서는 먼저 현행 영향평가 문헌을 비판적으로 검토한 뒤, 우리의 우려를 해소하는 새로운 접근법인 Scenario-Based Sociotechnical Envisioning을 제안한다."
    },
    {
      "section_title": "1 A Critique of Current Impact Assessment Methods",
      "text": "Power . Who is in charge of identifying and managing impacts? In other words, what are the underlying power relations in impact assessment? Impact assessment is political, and different impacts are prioritized according to the goals and social or economic priorities of the entity conducting the assessment. Whoever decides, also influences which impacts to look at, wh

- [title, abstract] developer 프롬프트

In [63]:
def build_developer_message_for_title_abstract() -> str:
    """
    Title + Abstract 전용 Developer Message
    - Title은 출력하지 않음 (호출 측이 \title{...}에 넣음)
    - Abstract '본문 LaTeX'만 산출
    """
    return (
        "입력은 다음 두 JSON 객체가 함께 제공된다:\n"
        "1) {'section_title': '<Title 텍스트>', 'text': '<보통 비어있음>'}\n"
        "2) {'section_title': 'ABSTRACT', 'text': '<초록 Markdown>'}\n\n"
        "출력 규칙:\n"
        "- Title은 출력하지 않는다(호출 측에서 \\title{...} 사용).\n"
        "- Abstract는 단락 하나 이상일 수 있으나, 문단 간에는 `\\\\` 를 사용한다.\n"
        "- 각 문단 시작은 \\noindent 를 붙인다.\n"
        "- 수식/표/링크/코드 등은 올바른 LaTeX으로 변환하되, 섹션 명령(\\section 등)과 환경(\\begin{abstract})은 추가하지 않는다.\n"
        "- 최종 출력은 'Abstract 본문 LaTeX'만 포함한다."
    )

- [global] developer 프롬프트

In [64]:
def build_developer_message_for_tex() -> str:
    prompt = '''입력은 항상 다음과 같은 구조의 JSON 객체로 주어진다:

{{'section_title': 'section_title_example', 'text': 'section_sentence_example'}}
여기서 "section_title"은 섹션의 제목을, "text"는 섹션의 본문 내용을 나타낸다.

사용하는 package는 다음과 같다.
\\input{math_commands.tex}
\\usepackage{iclr2025_conference, times}
\\usepackage{hyperref}
\\usepackage{url}
\\usepackage{graphicx}   % For figures
\\usepackage{booktabs}   % For professional tables
\\usepackage{tabularx}   % For tables with adjustable column width
\\usepackage{kotex}      % For Korean language support
\\usepackage{float}

규칙:

[섹션 제목 처리]
1. section_title이 "숫자"로만 시작하는 경우 (예: "1 INTRODUCTION"):
   - 숫자를 제거하고 LaTeX 섹션 명령어로 변환한다.
   - 출력 예: \\section{{INTRODUCTION}}

2. section_title이 "숫자.숫자" 형태로 시작하는 경우 (예: "3.1 PRELIMINARIES"):
   - 숫자 부분을 제거하고 LaTeX 서브섹션 명령어로 변환한다.
   - 출력 예: \\subsection{{PRELIMINARIES}}

3. section_title이 "숫자.숫자.숫자" 형태로 시작하는 경우 (예: "4.2.1 DETAILS"):
   - 숫자 부분을 제거하고 LaTeX subsubsection 명령어로 변환한다.
   - 출력 예: \\subsubsection{{DETAILS}}

4. 위 규칙에 해당하지 않으면 section_title을 그대로 LaTeX 섹션 명령어로 변환한다.
   - 출력 예: \\section{{ABSTRACT}}

[본문 처리]
1. text는 해당 섹션 제목 바로 아래에 출력한다.
2. 문단이 바뀔 때마다 문단 사이에 \\\\\\\\ (즉, 줄바꿈 명령어 4개)를 추가한다.
   - 반드시 `\\\\\\\\` 를 사용해 공백 하나가 들어가도록 한다.
3. 각 문단의 맨 앞 문장 시작 부분에는 항상 \\noindent를 추가한다.

[수식 처리]
1. text 안의 수식, 기호, 수학적 표현은 올바른 LaTeX 수식 문법으로 변환한다.
2. 짧은 수식은 \\( ... \\) 로 감싸 inline 수식으로 출력한다.
   - 예: "ˆ y" → \\( \\hat{{y}} \\)
   - 예: "N ∈ ℕ" → \\( N \\in \\mathbb{{N}} \\)
   - 예: "d_y" → \\( d_{{y}} \\)
3. 긴 수식(예: 다항식, 행렬, 연산자가 포함된 식 등)은 $$ ... $$ 로 감싸 block 수식으로 출력한다.
4. 원본 텍스트에 Markdown 스타일의 `$...$` 또는 `$$...$$`가 있으면 적절히 LaTeX 수식으로 변환한다.
5. 자연어 텍스트와 수식은 반드시 구분하여 출력한다.

[표 처리]
1. text 안의 Markdown 표 (| ... | 형태)는 반드시 LaTeX 표로 변환한다.
2. 표는 항상 \\begin{{table}}[H] ... \\end{{table}} 환경으로 출력한다.
3. 표 크기는 \resizebox{\textwidth}{!}{ ... } 를 쓰지 않고, 자연스럽게 표 크기를 만들고 \centering으로 가운데 정렬한다.
4. 표 내용은 tabular 환경을 사용하고, 열 개수는 원본 Markdown 표의 열 수를 기준으로 자동 생성한다.
5. 캡션은 Markdown 표 바로 위나 아래에 "Table N:" 으로 주어진 경우, 이를 추출하여 \\caption{{...}} 으로 넣는다.
6. \\label{{tab:...}} 는 캡션에서 추출한 번호를 기반으로 자동 부여한다.
7. 표의 \\begin{{table}}[H] ... \\end{{table}} 에는 \noindent를 붙이지 않는다.

[피겨 처리]
1. text 안의 Markdown 이미지 구문 (![Figure](./figures/...))은 반드시 LaTeX figure 환경으로 변환한다.
2. figure 환경은 항상 \\begin{{figure}}[H] ... \\end{{figure}} 형식을 사용한다.
3. 그림은 \\centering 으로 가운데 정렬한다.
4. 그림은 \\includegraphics[width=0.7\textwidth]{{...}} 형태로 삽입한다.
5. 캡션(\\caption{{...}})과 라벨(\\label{{...}})은 사용하지 않는다.
6. 결과적으로 단순히 그림만 논문에 포함되도록 한다.
7. 피겨의 \\begin{{figure}}[H] ... \\end{{figure}} 에는 \noindent를 붙이지 않는다.
'''

    return prompt

prompt = build_developer_message_for_tex()
print(prompt)

입력은 항상 다음과 같은 구조의 JSON 객체로 주어진다:

{{'section_title': 'section_title_example', 'text': 'section_sentence_example'}}
여기서 "section_title"은 섹션의 제목을, "text"는 섹션의 본문 내용을 나타낸다.

사용하는 package는 다음과 같다.
\input{math_commands.tex}
\usepackage{iclr2025_conference, times}
\usepackage{hyperref}
\usepackage{url}
\usepackage{graphicx}   % For figures
\usepackage{booktabs}   % For professional tables
\usepackage{tabularx}   % For tables with adjustable column width
\usepackage{kotex}      % For Korean language support
\usepackage{float}

규칙:

[섹션 제목 처리]
1. section_title이 "숫자"로만 시작하는 경우 (예: "1 INTRODUCTION"):
   - 숫자를 제거하고 LaTeX 섹션 명령어로 변환한다.
   - 출력 예: \section{{INTRODUCTION}}

2. section_title이 "숫자.숫자" 형태로 시작하는 경우 (예: "3.1 PRELIMINARIES"):
   - 숫자 부분을 제거하고 LaTeX 서브섹션 명령어로 변환한다.
   - 출력 예: \subsection{{PRELIMINARIES}}

3. section_title이 "숫자.숫자.숫자" 형태로 시작하는 경우 (예: "4.2.1 DETAILS"):
   - 숫자 부분을 제거하고 LaTeX subsubsection 명령어로 변환한다.
   - 출력 예: \subsubsection{{DETAILS}}

4. 위 규칙에 해당하지 않으면 section_title을 그대로 L

- user prompt

In [65]:
def build_user_prompt_title_abstract(title_obj: Dict, abstract_obj: Dict) -> str:
    """
    Title + ABSTRACT를 한 번에 전달하고, Abstract 본문만 변환해 달라고 지시.
    """
    payload = {
        "title": title_obj,
        "abstract": abstract_obj
    }
    return (
        "다음은 Title과 ABSTRACT 섹션의 JSON 입력이다. "
        "규칙에 따라 'Abstract 본문 LaTeX'만 출력하라. 문서/프리앰블/섹션 명령은 금지.\n\n"
        + json.dumps(payload, ensure_ascii=False, indent=2)
    )

def build_user_prompt_for_section(section_obj: Dict) -> str:
    """
    일반 섹션(ABSTRACT 제외): 규칙에 따라 섹션 제목 → \section/\subsection 매핑 + 본문 변환.
    출력은 하나의 LaTeX 블록(섹션 명령 + 본문)만 포함해야 하며,
    문서 프리앰블/abstract 환경 등은 포함하지 않는다.
    """
    return (
        "다음은 단일 섹션의 JSON 입력이다. 주어진 규칙([섹션 제목 처리], [본문 처리], [수식 처리], [표 처리])에 따라 "
        "해당 섹션의 LaTeX 블록을 생성하라. 문서/프리앰블/abstract 환경은 금지.\n\n"
        + json.dumps(section_obj, ensure_ascii=False, indent=2)
    )

In [66]:
def openai_tex_convert(client: OpenAI, dev_prompt, user_prompt: str) -> str:
    resp = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "developer", "content": [{"type": "text", "text": dev_prompt}]},
            {"role": "user", "content": [{"type": "text", "text": user_prompt}]},
        ],
        verbosity="low",
        reasoning_effort="low",
        store=False
    )
    return resp.choices[0].message.content.strip()

In [67]:
_LATEX_SPECIALS = {
    '\\': r'\textbackslash{}',  # 먼저 처리
    '{': r'\{',
    '}': r'\}',
    '$': r'\$',
    '&': r'\&',
    '#': r'\#',
    '_': r'\_',
    '%': r'\%',
    '^': r'\^{}',
    '~': r'\~{}',
}
def escape_latex(text: str) -> str:
    return "".join(_LATEX_SPECIALS.get(ch, ch) for ch in text)

TEMPLATE_PREAMBLE = r"""
\documentclass{article} % Don't change this

\input{math_commands.tex}

\usepackage{iclr2025_conference, times}
\usepackage{hyperref}
\usepackage{url}
\usepackage{graphicx}   % For figures
\usepackage{booktabs}   % For professional tables
\usepackage{tabularx}   % For tables with adjustable column width
\usepackage{kotex}      % For Korean language support
\usepackage{float}

% --- DOCUMENT METADATA ---
""".lstrip()

TEMPLATE_BEGIN_DOC = r"""
\date{}

% --- MAIN DOCUMENT ---
\iclrfinalcopy
\begin{document}
\maketitle
""".lstrip()

TEMPLATE_END_DOC = r"""
\end{document}
""".lstrip()

In [68]:
def assemble_tex_document(title_text: str,
                          abstract_body_tex: str,
                          section_tex_chunks: List[str]) -> str:
    title_tex = escape_latex(title_text)
    parts = []
    parts.append(TEMPLATE_PREAMBLE)
    parts.append(f"\\title{{\n{title_tex}\n}}\n")
    parts.append(TEMPLATE_BEGIN_DOC)

    # Abstract
    if abstract_body_tex.strip():
        parts.append("% --- ABSTRACT ---\n")
        parts.append("\\begin{abstract}\n")
        parts.append(abstract_body_tex.strip() + "\n")
        parts.append("\\end{abstract}\n\n")

    # Sections (LLM이 \section/\subsection 포함해서 반환)
    parts.append("% --- SECTIONS ---\n")
    for chunk in section_tex_chunks:
        parts.append(chunk.rstrip() + "\n\n")

    parts.append(TEMPLATE_END_DOC)
    return "".join(parts)

In [69]:
def md_to_full_iclr_tex(md_path: str) -> str:
    """
    1) md → 섹션 JSON
    2) Title + Abstract 한 번에 변환(본문만)
    3) 기타 모든 섹션은 개별 프롬프트로 변환(섹션 명령 포함)
    4) 최종 .tex 문자열로 조립
    """
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("환경변수 OPENAI_API_KEY가 설정되어야 합니다.")

    # 1) 섹션 JSON
    json_str = extract_all_sections_to_json(md_path)
    data = json.loads(json_str)
    sections: List[Dict] = data.get("sections", [])
    if not sections:
        raise RuntimeError("헤딩(#~######) 기반 섹션을 찾지 못했습니다.")

    # Title: 문서 첫 섹션의 제목을 사용
    title_obj = sections[0]
    doc_title = title_obj["section_title"].strip()

    # Abstract: 제목이 ABSTRACT(대소문자 무시)인 첫 섹션 찾기
    def _norm(s: str) -> str:
        s = re.sub(r'^\s*\d+(\.\d+)*\s*', '', s)  # 선행 번호 제거
        return s.strip().upper()

    abs_idx = next((i for i, s in enumerate(sections) if _norm(s["section_title"]) == "ABSTRACT"), None)
    abstract_obj = sections[abs_idx] if abs_idx is not None else {"section_title": "ABSTRACT", "text": ""}

    # 2) Title+Abstract 변환 (본문만 산출)
    client = OpenAI(api_key=api_key)
    up_ta = build_user_prompt_title_abstract(
        title_obj={"section_title": title_obj["section_title"], "text": title_obj.get("text", "")},
        abstract_obj={"section_title": abstract_obj["section_title"], "text": abstract_obj.get("text", "")},
    )
    dev_ta = build_developer_message_for_title_abstract()
    abstract_body_tex = openai_tex_convert(client, dev_ta, up_ta)

    # 3) 나머지 섹션(Title, Abstract 제외) 개별 변환
    dev_sec = build_developer_message_for_tex()
    section_tex_chunks: List[str] = []
    for i, sec in enumerate(sections):
        if i == 0:
            continue  # Title은 \title로만 사용
        if abs_idx is not None and i == abs_idx:
            continue  # Abstract는 이미 처리

        user_prompt = build_user_prompt_for_section({
            "section_title": sec["section_title"],
            "text": sec.get("text", "")
        })
        chunk_tex = openai_tex_convert(client, dev_sec, user_prompt)
        section_tex_chunks.append(chunk_tex)

    # 4) 최종 조립
    return assemble_tex_document(doc_title, abstract_body_tex, section_tex_chunks)

In [70]:
tex_str = md_to_full_iclr_tex(md_path='output.md')
out_path = Path("paper.tex").expanduser()
out_path.write_text(tex_str, encoding="utf-8")
print(f"✅ LaTeX 파일 생성 완료: {out_path}")

✅ LaTeX 파일 생성 완료: paper.tex
